### 경제 뉴스 추천 시스템 DB

In [1]:
import pandas as pd
import datetime
try:
    import pymysql
except:
    raise Exception('you need to install pymysql\n$ : python -m pip install pymysql')
import traceback, json, re

"""
사용할 라이브러리 : pymysql, pandas

요구사항
1. 클래스 내 생성자, 소멸자, insert, select 함수 구현
2. 테스트를 위한 실행 코드 작성

참고해볼만한 코드
https://github.com/devsosin/sosin/blob/main/sosin/databases/rdb.py
"""

class NewsDB:
    """
    클래스 설명
    """

    def __init__(self, db_config: str, category_file: str|dict, sql_file: str|None=None) -> None:
        """
        데이터베이스 접속
        인자 : 데이터베이스 접속정보
        """
        # 데이터베이스에서 select 해서 가져와도 상관없음.

        # 계정 정보 파일에서 필요 인자 딕셔너리로 생성
        # 딕셔너리 언패킹하여 인자 값 할당 후 서버 연동
        with open(db_config) as f:
            res = dict(map(lambda x: x.replace('\n','').split('='), f.readlines()))
            for i, v in res.items():
                if i=="password":
                    pass
                elif v.isdigit(): res[i] = int(v)
        # 로그인 위한 인자명 출력
        print(res.keys())
        # 딕셔너리 언패킹하여 인자 값 할당 후 서버 연동
        self.remote = pymysql.connect(**res)
        #self.remote = pymysql.connect(**db_config)

        if category_file:
            if type(category_file)==str:
                if category_file[-5:]=='.json':
                    with open(category_file, 'r') as f:
                        self.SUB_CATEGORY_DICT = json.load(f)
                else:
                    raise Exception('category_file can only use .json!\nuse .json or insert dictionary')
            elif type(category_file)==dict:
                    self.SUB_CATEGORY_DICT = category_file
            else:
                raise Exception('category_file can only use .json and dictionary!\ninsert path of .json or dictionary')

        else:
            with self.remote.cursor() as cur:
                cur.execute('select * from CATEGORY')
                tmp = cur.fetchall()

            self.SUB_CATEGORY_DF = pd.DataFrame(tmp, columns=['cat2_id', 'cat1_name', 'cat2_name', 'platform_name'])
            self.SUB_CATEGORY_DICT = {}
            [self.SUB_CATEGORY_DICT.update({p: {}}) for p in self.SUB_CATEGORY_DF.platform_name.unique()]
            [self.SUB_CATEGORY_DICT[p].update({c1: {}}) for p in self.SUB_CATEGORY_DF.platform_name.unique() for c1 in self.SUB_CATEGORY_DF[self.SUB_CATEGORY_DF.platform_name==p].cat1_name.unique()]
            [self.SUB_CATEGORY_DICT[p][c1].update({c2[1]: c2[0]}) for p in self.SUB_CATEGORY_DF.platform_name.unique() for c1 in self.SUB_CATEGORY_DF[self.SUB_CATEGORY_DF.platform_name==p].cat1_name.unique() for c2 in self.SUB_CATEGORY_DF[(self.SUB_CATEGORY_DF.platform_name==p)&(self.SUB_CATEGORY_DF.cat1_name==c1)][['cat2_id', 'cat2_name']].values]

        # 테이블 생성. if not exists로 오류 해결.
        if sql_file:
            with self.remote.cursor() as cur:
                # 파일에서 '\ufeff'가 읽힐 경우 encoding하거나 replace로 제거
                with open(sql_file, 'r', encoding='utf-8-sig') as f :
                    # split(';')이기에 마지막에 ['']가 존재하여 [:-1]로 슬라이싱
                    commands = f.read().split(';')[:-1]
                for command in commands:
                    cur.execute(command.strip())


        # category 테이블 record 적재.
        if category_file:
            with self.remote.cursor() as cur:
                # category 불러오기
                if type(category_file)==str:
                    try:
                        with open(category_file, 'r') as f:
                            category = json.load(f)
                    except:
                        raise Exception("category.json don't exist or path is worng.")

                # category 확인 및 적재
                done_list = []
                error_list = []
                try:
                    cur.execute('select count(*) from CATEGORY')
                    if cur.fetchall()[0][0]==145:
                        print('='*50)
                        print('already all record is loaded on CATEGORY table')
                        pass
                    else:
                        for platfrom in category.keys():
                            for cat_1 in category[platfrom].keys():
                                for name, id in category[platfrom][cat_1].items():
                                    try:
                                        cur.execute(f"insert into CATEGORY values({id}, '{cat_1}','{name}', '{platfrom}')")
                                        done_list.append([id, cat_1, name, platfrom])
                                    except:
                                        error_list.append([id, cat_1, name, platfrom])
                                        print(f"already values({id}, '{cat_1}','{name}', '{platfrom}' exist")
                        print('='*50)
                        print(f'done_task: {len(done_list)}, error_task: {len(error_list)}')
                except:
                    raise Exception("make tables first!")

            if category_file:
                with self.remote.cursor() as cur:
                    cur.execute('select * from CATEGORY')
                    tmp = cur.fetchall()
                self.SUB_CATEGORY_DF = pd.DataFrame(tmp, columns=['cat2_id', 'cat1_name', 'cat2_name', 'platform_name'])

        # DML은 별도 commit 필요!
        self.remote.commit()
        print('task is done!')

    def __del__(self) -> None:
        """
        데이터베이스 연결 해제
        """
        self.remote.close()

    def insert_news(self, df: pd.DataFrame) -> None:
        """
        인자 : 뉴스기사 데이터프레임
        columns = ['cat1_name', 'cat2_name', 'platform_name', 'title', 'press', 'writer', 'date_upload', 'date_fix', 'content', 'sticker', 'url']

        우선 데이터프레임의 column명 체크하여 News 테이블의 칼럼이름과 일치하지 않을 경우 에러 발생시키기

        insert SQL문 생성
        execute 대신 execute_many 메서드로 한번에 삽입

        1. 플랫폼 정보 id로 변환
        2. 메인카테고리 숫자로 변환
        3. 서브카테고리 숫자로 변환
        4. DB에 적재

        """

        # column 이름 일치 확인
        df_columns = ['cat1_name', 'cat2_name', 'platform_name', 'title', 'press', 'writer', 'date_upload', 'date_fix', 'content', 'sticker', 'url']
        df = df[df_columns]
        for col in df.columns:
            df.loc[df[col].isna(), col] = None
            df.loc[(df[col]==''), col] = None

        # platform_name 변환 및 cat2_id 할당
        df['platform_name'] = [platform if platform in ("네이버", "다음") else "네이버" if platform.upper()=="NAVER" else "다음" if platform.upper()=="DAUM" else None for platform in df['platform_name']]
        if df['platform_name'].isna().any():
            raise Exception('some platform_name rows wrong!!')

        df['cat2_id'] = [self.SUB_CATEGORY_DICT[_[0]][_[1]][_[2]] for _ in df[['platform_name', 'cat1_name', 'cat2_name']].values]
        df_columns = ['cat2_id']+df_columns
        df = df[df_columns]
        df.drop(columns=['cat1_name', 'cat2_name', 'platform_name'], inplace=True)

        # sticker json 따옴표 변경
        if not sum(df['sticker'].apply(str).str.count('"')):
            df['sticker'] = df['sticker'].apply(json.dumps)

        with self.remote.cursor() as cur:
            my_query = "insert ignore into NEWS(cat2_id, title, press, writer, date_upload, date_fix, content, sticker, url) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cur.executemany(my_query, df.values.tolist())
        self.remote.commit()
        print('inserted news!')

    def change_comment_df(self, df: pd.DataFrame([list, str])) -> None:
        """
        인자 : 댓글 데이터프레임
        columns = ['user_id', 'user_name']

        데이터프레임 칼럼 체크하여 Comment 테이블의 칼럼과 일치하지 않을 경우 에러

        1. 유저 테이블에서 있는지 체크, id값 있을 경우 변환
        2. 신규 유저일 경우 유저 테이블에 추가, id값 가져오기 (DB에 유저 정보가 저장되어있다면 가져오기)
        3. url을 통해 코멘트 별 뉴스기사 id 가져오기 (select)
        """

        # user_df 생성 및 적재
        with self.remote.cursor() as cur:
            my_query = "insert ignore into USER(user_id, user_name) values(%s, %s)"
            cur.executemany(my_query, df.values.tolist())
        self.remote.commit()

        # get user_id
        with self.remote.cursor() as cur:
            cur.execute("select id, user_id from USER")
            user_df = pd.DataFrame(cur.fetchall(), columns=['id', 'user_id'])
        df = user_df[user_df.user_id.isin(df.user_id.values)]

        # get news_id
        with self.remote.cursor() as cur:
            cur.execute("select c.user_id, c.news_id, c.comment, n.url from COMMENT c, NEWS n where c.news_id = n.id")
            news_df = pd.DataFrame(cur.fetchall(), columns=['id', 'news_id', 'comment', 'url'])
        df = news_df[news_df.id.isin(df.id.values)]
        return df



    def insert_comment(self, df: pd.DataFrame) -> None:
        """
        인자 : 댓글 데이터프레임
        columns = ['user_id', 'user_name', 'comment', 'date_upload', 'date_fix', 'good_cnt', 'bad_cnt', 'url']

        데이터프레임 칼럼 체크하여 Comment 테이블의 칼럼과 일치하지 않을 경우 에러

        1. 댓글 id로 변환하는 함수 호출하여 변환한 데이터프레임 가져오기
        2. DB에 적재
        """

        # column과 url 확인
        df_columns = ['user_id', 'user_name', 'comment', 'date_upload', 'date_fix', 'good_cnt', 'bad_cnt', 'url']
        df = df[df_columns]
        for col in df.columns:
            df.loc[df[col].isna(), col] = None
            df.loc[(df[col]==''), col] = None

        # get news_id
        with self.remote.cursor() as cur:
            cur.execute("select id, url from NEWS")
            tmp_df = pd.DataFrame(cur.fetchall(), columns=['news_id', 'url'])

        df = pd.merge(df, tmp_df, 'left', 'url').reset_index(drop=True)
        del tmp_df

        # comment_df 변환
        df = df[~df.user_id.isna()].reset_index(drop=True)
        df_columns.pop()
        df_columns = ['news_id']+df_columns
        df = df[df_columns]

        # user_df 생성 및 적재
        user_df = df.groupby(['user_id', 'user_name']).count().reset_index()[['user_id', 'user_name']]
        with self.remote.cursor() as cur:
            my_query = "insert ignore into USER(user_id, user_name) values(%s, %s)"
            cur.executemany(my_query, user_df.values.tolist())
        self.remote.commit()
        print('inserted user!')

        # get user_id
        with self.remote.cursor() as cur:
            cur.execute("select id, user_id from USER")
            tmp_df = pd.DataFrame(cur.fetchall(), columns=['id', 'user_id'])

        # comment_df 변환 및 적재
        df_columns.pop(2)
        df = pd.merge(df, tmp_df.drop_duplicates('user_id').reset_index(drop=True), 'left', 'user_id').reset_index(drop=True)
        df = df.drop(columns='user_id').rename(columns={'id': 'user_id'})[df_columns]
        del user_df,

        with self.remote.cursor() as cur:
            my_query = "insert ignore into COMMENT(news_id, user_id, comment, date_upload, date_fix, good_cnt, bad_cnt) values(%s, %s, %s, %s, %s, %s, %s)"
            cur.executemany(my_query, df.values.tolist())
        self.remote.commit()
        print('inserted comment!')


    # 각 인원이 ERD 통해 데이터베이스에 테이블 생성해서 수집한 데이터로 테스트해 볼 것

    ## 강사님 코드
    ## 프로젝트 중이나 종료 후 여유될 때 만들어볼 것.
    def select_news(self, start_date=None, end_date=None,
                    platform: str|None=None, category1: str|list|None=None, category2: str|list|None=None
                    , columns_name: list=['id', 'cat2_id', 'title', 'press', 'writer', 'date_upload', 'date_fix', 'content', 'sticker', 'url'],
                    limit: int|str|None=None) -> pd.DataFrame:
        """
        인자 : 데이터를 꺼내올 때 사용할 parameters
        (어떻게 검색(필터)해서 뉴스기사를 가져올 것인지)

        DB에 들어있는 데이터를 꺼내올 것인데, 어떻게 꺼내올지를 고민

        인자로 받은 파라미터 별 조건을 넣은 select SQL문 작성

        SQL문에 추가할 내용들
        1. 가져올 칼럼
        2. JOIN할 경우 JOIN문 (플랫폼, 카테고리)
        3. WHERE 조건문
        4. LIMIT, OFFSET 등 처리
        """



        where_sql = []

        if start_date and end_date:
            where_sql.append(f"date_upload BETWEEN '{start_date}' AND '{end_date}'")
        elif start_date:
            where_sql.append(f"date_upload >= '{start_date}'")
        elif end_date:
            where_sql.append(f"date_upload <= '{end_date}'")

        if platform or category1 or category2:
            tmp_SUB_CATEGORY_DF = self.SUB_CATEGORY_DF.copy()
            if platform:
                if platform=='다음':
                    where_sql.append(f"cat2_id<20000")
                elif platform=='네이버':
                    where_sql.append(f"cat2_id>20000")
                else:
                    raise Exception('you can use only "다음" or "네이버"!')
                tmp_SUB_CATEGORY_DF = tmp_SUB_CATEGORY_DF[self.SUB_CATEGORY_DF.platform_name==platform]

            if category1:
                isin_list = []
                if type(category1)==str:
                    category1 = [category1]
                for value in category1:
                    isin_list.append(value)
                tmp_SUB_CATEGORY_DF = tmp_SUB_CATEGORY_DF[tmp_SUB_CATEGORY_DF.cat1_name.isin(isin_list)]

            if category2:
                isin_list = []
                if type(category2)==str:
                    category2 = [category2]
                for value in category2:
                    isin_list.append(value)
                tmp_SUB_CATEGORY_DF = tmp_SUB_CATEGORY_DF[tmp_SUB_CATEGORY_DF.cat2_name.isin(isin_list)]
            where_sql.append(f"cat2_id in ({','.join(tmp_SUB_CATEGORY_DF.cat2_id.apply(str).values.tolist())})")
        else:
            tmp_SUB_CATEGORY_DF=None


        # main_query = f'SELECT id,cat2_id,title,press,writer,date_upload,content,sticker,url FROM NEWS '
        main_query = f'SELECT {", ".join(columns_name)} FROM NEWS '

        final_result = []
        if where_sql:
            main_query += f' WHERE {" AND ".join(where_sql)}'
        if int(limit)<100000:
            main_query += f' limit {limit}'
            with self.remote.cursor() as cur:
                cur.execute(main_query)
                result = cur.fetchall()
                final_result.extend(result)
        else:
            # 1GB Ram 제한 (limit, offset)
            pagination_sql = ' LIMIT 100000 OFFSET {}'
            offset = 0
            while True:
                with self.remote.cursor() as cur:
                    cur.execute(main_query + pagination_sql.format(offset))
                    result = cur.fetchall()
                    final_result.extend(result)

                if len(result) < 100000:
                    break

                offset += 100000 # LIMIT

        df = pd.DataFrame(final_result, columns=columns_name)
        if 'cat2_id' in columns_name:
            tmp_SUB_CATEGORY_DF = self.SUB_CATEGORY_DF[self.SUB_CATEGORY_DF.cat2_id.isin(df.cat2_id.unique())]
            df = pd.merge(df, tmp_SUB_CATEGORY_DF, 'left', 'cat2_id')
            df = df[['cat2_id', 'cat1_name', 'cat2_name', 'platform_name', 'title', 'press', 'writer', 'date_upload', 'content', 'sticker', 'url']]

        return df

    def select(self, query_command: str) -> tuple:
        """
        인자 : 데이터를 꺼내올 때 사용할 parameters
        (어떻게 검색(필터)해서 뉴스기사를 가져올 것인지)

        DB에 들어있는 데이터를 꺼내올 것인데, 어떻게 꺼내올지를 고민

        인자로 받은 파라미터 별 조건을 넣은 select SQL문 작성

        SQL문에 추가할 내용들
        1. 가져올 칼럼
        2. JOIN할 경우 JOIN문 (플랫폼, 카테고리)
        3. WHERE 조건문
        4. LIMIT, OFFSET 등 처리
        """
        if query_command.find(';')>=0:
            raise Exception('you can use only one query')
        with self.remote.cursor() as cur:
            cur.execute(query_command)
            res = cur.fetchall()
        return res

    def select_user(self) -> pd.DataFrame:
        """
        인자 : 데이터를 꺼내올 때 사용할 parameters
        (어떻게 검색(필터)해서 유저를 가져올 것인지)

        SQL문에 추가할 내용들
        1. 가져올 칼럼
        2. JOIN할 경우 JOIN문
        3. WHERE 조건문
        4. LIMIT, OFFSET 등 처리
        """
        with self.remote.cursor() as cur:
            cur.execute('select * from USER')
            res = pd.DataFrame(cur.fetchall(), columns=['id', 'user_id', 'user_name'])
        return res

    def select_comment(self) -> pd.DataFrame:
        """
        인자 : 데이터를 꺼내올 때 사용할 parameters
        (어떻게 검색(필터)해서 댓글을 가져올 것인지)

        SQL문에 추가할 내용들
        1. 가져올 칼럼
        2. JOIN할 경우 JOIN문 (유저정보를 같이 가져올 경우)
        3. WHERE 조건문
        4. LIMIT, OFFSET 등 처리
        """
        with self.remote.cursor() as cur:
            cur.execute('select * from COMMENT')
            res = pd.DataFrame(cur.fetchall(), columns=['id', 'news_id', 'user_id', 'comment', 'date_upload', 'date_fix', 'good_cnt', 'bad_cnt'])
        return res

---
local db 연결

In [2]:
def read_config(p:str) -> dict:
    """
    p: config file 경로
    """
    with open('db.config', 'r') as f:
        lines = f.readlines()

    config_dict={}

    for l in lines:
        # k, v = l.split('=') # 일반적인 경우 다른 '='값이 없을 경우
        idx = l.index('=') # config파일에 다른 '=' 값이 존재할 경우 대비
        k = l[:idx]
        v = l[idx+1:]
        config_dict[k] = v.strip()


    # with open(db_config) as f:
    # db_config = dict(map(lambda x: x.replace('\n','').split('='), f.readlines()))
    # for i, v in db_config.items():
    #     if v.isdigit(): db_config[i] = int(v)

    return config_dict 

db_config = read_config('db.config')
remote = pymysql.connect(**db_config)

In [3]:
#DB연결
try:
    DB = NewsDB("./db.config","./category.json", "./Elementary_ERD.sql")
    print('데이터베이스 연결 성공')
except:
    print('데이터베이스 연결 실패')

dict_keys(['host', 'user', 'password', 'database'])
already all record is loaded on CATEGORY table
task is done!
데이터베이스 연결 성공


---


뉴스, 댓글 데이터 적재하기

In [33]:
#다음 뉴스 insert
daum_news=pd.read_csv("./daum_economy_filtered_3month.csv")
daum_news["date_fix"]=pd.to_datetime(daum_news["date_fix"])
daum_news["date_upload"]=pd.to_datetime(daum_news["date_upload"])
daum_news["writer"]=daum_news["writer"].fillna("")
daum_news["content"]=daum_news["content"].fillna(daum_news["title"])
DB.insert_news(daum_news)

#네이버 뉴스 insert
naver_news=pd.read_csv("./naver_economy_no_comment.csv")
naver_news["date_fix"]=pd.to_datetime(naver_news["date_fix"])
naver_news["date_upload"]=pd.to_datetime(naver_news["date_upload"])
naver_news["writer"]=naver_news["writer"].fillna("")
naver_news["content"]=naver_news["content"].fillna(naver_news["title"])
DB.insert_news(naver_news) #commment 열 삭제

#댓글 insert
comment=pd.read_csv("./naver_economy_comment.csv")
comment["date_fix"]=pd.to_datetime(comment["date_fix"])
comment["date_upload"]=pd.to_datetime(comment["date_upload"])
DB.insert_comment(comment)


# select 테스트 (뉴스, 코멘트, 유저)

inserted news!


---

데이터 조회

In [ ]:
# #DB연결
# try:
#     DB = NewsDB("./secret_db_local.config","./category.json", "./Elementary_ERD.sql")
#     print('데이터베이스 연결 성공')
# except:
#     print('데이터베이스 연결 실패')

In [12]:
# 뉴스 데이터 조회 (return: dataframe)
news_df=pd.DataFrame(DB.select(f"SELECT * FROM NEWS WHERE date_upload >= '2023-9-12' "))
news_df.columns = ['id', 'cat2_id', 'title', 'press', 'writer', 'date_upload', 'date_fix', 'content', 'sticker', 'url']
news_df.tail(3)

id  cat2_id                               title   press writer  \
21217  1210624    10315          보잉,베트남에서 10조원 상당 737제트기 수주    한국경제    김정아   
21218  1210625    10315   북-러 수일내 정상회담…"김정은 열차타고 러 이동중"(종합)    이데일리    김상윤   
21219  1210626    10315  유럽 태양광업체들 파산 위기...中 모듈 재고만 2년 수요규모  파이낸셜뉴스    송경재   

              date_upload             date_fix  \
21217 2023-09-12 00:03:00  0000-00-00 00:00:00   
21218 2023-09-12 04:25:00  2023-09-12 04:32:00   
21219 2023-09-12 04:35:00  0000-00-00 00:00:00   

                                                 content  \
21217  보잉(BA)은 베트남항공으로부터 80억달러(10조6,500억원) 상당의 737맥스 ...   
21218  [뉴욕=이데일리 김상윤 특파원, 이소현 기자] 러시아 크렘링궁은 푸틴 러시아 대통령...   
21219  [파이낸셜뉴스] 유럽 태양광 업계가 태양광 발전을 통해 에너지 자립을 달성하고, 탄...   

                                                 sticker  \
21217  "{'LIKE': 0, 'SAD': 0, 'ANGRY': 0, 'RECOMMEND'...   
21218  "{'LIKE': 1, 'SAD': 0, 'ANGRY': 0, 'RECOMMEND'...   
21219  "{'LIKE': 0, 'SAD': 0, 'ANGRY': 0, 'RECOMMEND'...   

                                          url  
21217  https://v.daum.net/v/20230912000303574  
21218  https://v.daum.net/v/20230912042543668  
21219  https://v.daum.net/v/20230912043500700

In [5]:
#댓글 데이터 조회
comment_df=DB.select_comment()
comment_df.tail(3)

id  news_id  user_id                              comment  \
1928755  1928756   488304     6526                              삼전 떡상이네   
1928756  1928757   870045   314217                         식량위기에 직면한 유럽   
1928757  1928758   870046   294379  이번에는 제발 파월  쓸데없는 말않고 제대로상승장오길  바랍니다   

                date_upload            date_fix  good_cnt  bad_cnt  
1928755 2023-06-13 05:10:22 2023-06-13 05:10:22         1        2  
1928756 2023-06-13 02:32:32 2023-06-13 02:32:32         0        0  
1928757 2023-06-13 00:46:22 2023-06-13 00:46:22         2        4

In [10]:
#유저 데이터 조회
user_df=DB.select_user()
user_df.head(3)

id user_id user_name
0   1   1002O  thsu****
1   2   1007C  real****
2   3   100IJ  kiul****

---

### 특정 user가 댓글 남긴 기사들 중 하나 랜덤 추출 → 유사한 기사 가져오기

In [31]:
import random

def get_random_news_url_by_common_user():
    comment_cnt = comment_df.groupby("user_id").count()
    common_user = comment_cnt.sort_values(by=["id"], ascending=False)[:10]    # 댓글 많이 남긴 user 10명
    user_comments = comment_df[comment_df["user_id"] == common_user.index[random.randint(0,9)]]   # 10명 중 한 명 user_id 추출
    random_comment = user_comments.sample(n=1)
    news_id = random_comment.iloc[0]["news_id"]
    url=pd.DataFrame(DB.select(f"SELECT * FROM NEWS where id = {news_id}"))[9].values.astype('str')[0]
    return url
url = get_random_news_url_by_common_user()
url

'https://n.news.naver.com/mnews/article/016/0002168575?sid=101'

In [7]:
from modeling import load_data, load_doc2vec, show_similar_results

In [ ]:
data = load_data()
model = load_doc2vec()

In [32]:
data[data['url'] == url][['title', 'date_upload', 'content']]

title          date_upload  \
224713  93만명 청약광풍 ‘흑석자이’ 매매·전세 수억 반등  2023-07-12 11:32:00   

                                                  content  
224713   “흑석자이 전세 가격이 전용면적 59㎡가 7억원을 찍는 등 반등세입니다.”(흑석동...

In [33]:
show_similar_results(data=data, model=model, url=url)

title  \
225023     [르포] 93만명 청약광풍 이유 있었네…흑석 집값·전세 수억대 반등 [부동산360]   
207579             파리 날릴 때는 가격 낮추기 바쁘더니…얼굴 바꾼 보류지[부동산360]   
302293                       화성 동탄에 무슨일이?…3가구 줍줍에 8천명 몰렸다   
214319             “차기 반포 대장아파트 원베일리...사전점검 후 전셋값 1억원 상승”   
207817                          4억~5억 상승 속출...‘마용성’ 또 오른다   
133835                                  백광산업 “前 대표 구속 맞다”   
207229                       착한 분양가 내세운 ‘첨단 제일풍경채’ 이번주 분양   
107821                     중소기업유통센터, 광복절 맞아 독립유공자 후손에 기부금   
221523                       잠실의 힘…송파구 올 상승률, 서울 유일 '플러스'   
279401  “5개월 만에 5억 뛰었다”...‘강남 원조 부촌’ 방배의 질주 [김경민의 부동산NOW]   

                date_upload  \
225023  2023-07-11 14:19:00   
207579  2023-09-12 06:00:00   
302293  2023-08-16 14:49:00   
214319  2023-08-18 11:18:00   
207817  2023-09-11 11:18:00   
133835  2023-07-21 11:17:00   
207229  2023-09-13 07:01:00   
107821  2023-08-16 09:00:00   
221523  2023-07-23 17:32:00   
279401  2023-09-01 22:02:00   

                                                  content  cos_simil  
225023   [헤럴드경제=고은결·이준태 기자] “흑석자이 전세 가격이 전용면적 59㎡가 7억원...   0.857516  
207579   [헤럴드경제=고은결 기자] 부동산 시장 침체에 외면받던 아파트 보류지 매물이 다시...   0.568966  
302293   16일 한국부동산원 청약홈에 따르면 태영동탄 컨소시엄이 동탄2신도시 신주거문화타운...   0.568176  
214319   “원베일리는 입주장과 관계 없이 가격이 오를 수밖에 없습니다. 최근 전셋값도 올라...   0.559864  
207817   최근 수년간 신흥 주거 지역으로 꼽히며 집값이 폭등했던 마용성(마포·용산·성동구)...   0.553377  
133835   백광산업은 지난 17일 한국거래소의 조회공시 요구에 대해 "김모 전 대표는 특정경...   0.552573  
207229    제일건설이 9월에 분양예정인 광주 ‘첨단 제일풍경채’가 올해 광주지역 민영분양 ...   0.550133  
107821   (서울=연합뉴스) 박상돈 기자 = 중소기업유통센터는 광복절을 맞아 지난 14일 독...   0.549452  
221523   올 들어 서울 아파트 가격 회복을 주도하는 강남4구(강남·서초·송파·강동) 중 송...   0.549390  
279401   올 들어 ‘집값 바닥론’이 확산되면서 서울 서초구 방배동 일대 아파트값이 상승세를...   0.548414